In [4]:
from arcticdb import Arctic
ac = Arctic("lmdb:///Users/baeyeongmin/arcticdb")
DELTA_DB_PATH='/Users/baeyeongmin/deltadb'
date='20230915'

In [5]:
import os
from glob import glob
logs_path = os.path.join(DELTA_DB_PATH, date, "ticks", "*.csv")
paths = glob(logs_path)

In [6]:
import pandas as pd
import polars as pl

In [12]:
ORDERBOOK_COLS = [
 'offerrem2',
 'offerho4',
 'bidho5',
 'offerho3',
 'offerrem3',
 'bidho4',
 'offerrem4',
 'offerho6',
 'bidho7',
 'offerho5',
 'offerrem5',
 'bidho6',
 'offerho8',
 'bidho9',
 'offerho7',
 'bidho8',
 'offerrem1',
 'offerho9',
 'offerrem6',
 'offerrem7',
 'offerrem8',
 'offerrem9',
 'bidrem3',
 'bidrem4',
 'bidrem1',
 'bidrem2',
 'bidrem9',
 'bidho1',
 'bidrem7',
 'bidrem8',
 'hotime',
 'offerho2',
 'bidho3',
 'bidrem5',
 'offerho1',
 'bidho2',
 'bidrem6',
 'bidrem10',
 'bidho10',
 'shcode',
 'offerho10',
 'offerrem10'
]

In [10]:
df = pd.read_csv(paths[0], usecols=ORDERBOOK_COLS, index_col=['local_timestamp'])

In [9]:
# df = pl.read_csv(paths[0], try_parse_dates=True, infer_schema_length=10_000)

In [13]:
df = df[ORDERBOOK_COLS]

In [14]:
df = df.rename(columns={
    col: col.replace("bidho", "bpx").replace("bidrem", "bqty").replace("offerho", "apx").replace("offerrem", "aqty").replace('shcode', 'symbol').replace('hotime', 'exchange_timestamp')
    for col in df.columns
})
df = df[sorted(df.columns)]

In [15]:
df

,apx1,apx10,apx2,apx3,apx4,apx5,apx6,apx7,apx8,apx9,...,bqty2,bqty3,bqty4,bqty5,bqty6,bqty7,bqty8,bqty9,exchange_timestamp,symbol
local_timestamp,,,,,,,,,,,,,,,,,,,,,
1.694740e+09,34435,34480,34440,34445,34450,34455,34460,34465,34470,34475,...,11746,17464,2362,274,5865,63,15613,10624,95730,069500
1.694740e+09,12540,12630,12550,12560,12570,12580,12590,12600,12610,12620,...,726,1059,2982,281,301,400,2245,4586,95730,078520
1.694740e+09,3095,3140,3100,3105,3110,3115,3120,3125,3130,3135,...,177,3343,24729,12327,14656,10182,65019,8782,95730,088350
1.694740e+09,2235,2280,2240,2245,2250,2255,2260,2265,2270,2275,...,12,77,17,475,2361,2244,9010,2008,95730,088790
1.694740e+09,12280,12370,12290,12300,12310,12320,12330,12340,12350,12360,...,6020,7497,2773,1462,1319,5747,4864,9656,95730,088980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.694742e+09,18445,0,18450,18455,19750,23000,0,0,0,0,...,0,0,0,0,0,0,0,0,104335,580023
1.694742e+09,12895,13100,12900,12905,12910,12915,12920,12925,12930,12935,...,125,20054,20132,21409,20000,20002,20002,20010,104335,580037
1.694742e+09,19655,22000,19660,19665,19670,19675,20000,20250,20750,21500,...,1,0,0,0,0,0,0,0,104335,610012


In [16]:
df.index = pd.to_datetime(df.index, utc=True, unit='s')

In [17]:
today = df.index[0].date()

In [18]:
from datetime import datetime

In [19]:
def combine_date_time(date_obj, time_str):
    time_str = f"{time_str:06}"
    hour = int(time_str[0:2])
    minute = int(time_str[2:4])
    second = int(time_str[4:6])
    return datetime(date_obj.year, date_obj.month, date_obj.day, hour, minute, second)


In [20]:
df['exchange_timestamp'] = df['exchange_timestamp'].apply(lambda x: combine_date_time(date_obj=today, time_str=x))

In [21]:
if 'ebest.orderbook' not in ac.list_libraries():
    # library does not already exist
    ac.create_library('ebest.orderbook')
lib = ac.get_library('ebest.orderbook')

In [23]:
from tqdm import tqdm

In [24]:
symbols = df['symbol'].unique()
for symbol in tqdm(symbols):
    orderbook_df = df[df['symbol'] == symbol].copy()
    orderbook_df.sort_index(inplace=True)
    lib.write(symbol, orderbook_df)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2063/2063 [02:17<00:00, 15.00it/s]


array(['069500', '078520', '088350', ..., '500078', '448310', '570063'],
      dtype=object)